In [6]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import shutil
from label_pipeline import InteriorImageClassifier, generate_class_label2ref_images
from config import MODEL_NAME, BASE_PROMPT_EN, BASE_PROMPT_RU

In [2]:
def process_directory(
        classifier: InteriorImageClassifier,
        image_dir: Path,
        output_dir: Path,
        ref_images_dir: Path | None = None,
        extensions: tuple = (".jpg", ".png", ".jpeg")
    ) -> None:
    """
    Обрабатывает изображения и сортирует по папкам классов
    с постепенной записью результатов в CSV.
    """
    output_dir.mkdir(parents=True, exist_ok=True)  # Создаем корневую директорию для результатов
    
    # Обрабатываем каждое изображение
    img_files = [f for f in image_dir.iterdir() if f.suffix.lower() in extensions]
    for img_file in tqdm(img_files, desc="Обработка изображений"):
        class_label2ref_images = None
        if ref_images_dir is not None:
            class_label2ref_images = generate_class_label2ref_images(
                ref_images_dirpath=ref_images_dir,
                max_files_per_class=1
            )
        image = Image.open(img_file)
        result = classifier(image, class_label2ref_images=class_label2ref_images)
        
        class_dir = output_dir / result["class"]  # Создаем папку класса (A0, A1... ERROR)
        class_dir.mkdir(exist_ok=True)
        
        shutil.copy2(img_file, class_dir / img_file.name)  # Копируем изображение в папку класса
    
    print(f"\nГотово! Результаты в {output_dir}")

In [3]:
# Инициализация классификатора
classifier = InteriorImageClassifier(model_name=MODEL_NAME, base_prompt=BASE_PROMPT_EN)
print(classifier.pipe.torch_dtype)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Device set to use cuda:0


torch.bfloat16


In [4]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"

image_dir = data_dir / "cosmetic/cosmetic+"
output_dir = data_dir / "labeled_with_few_shot"
ref_images_dir = data_dir / "reference_images_resized"

Директория: /home/little-garden/CodeProjects/InteriorClass


In [5]:
process_directory(
    classifier=classifier,
    image_dir=image_dir,
    output_dir=output_dir,
    ref_images_dir=ref_images_dir
)

Обработка изображений: 100%|██████████| 13426/13426 [5:01:55<00:00,  1.35s/it]  


Готово! Результаты в /home/little-garden/CodeProjects/InteriorClass/data/labeled_with_few_shot
